In [1]:
import tensorflow as tf

# Layers and models

Most models are made of layers. Layers are functions with a known mathematical structure that can be reused and have trainable variables. In TensorFlow, most high-level implementations of layers and models, such as Keras or Sonnet, are built on the same foundational class: tf.Module.

In [7]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5., name='train_me')
        self.non_trainable_variable = tf.Variable(5., trainable=False,
                                                   name='do_not_train_me')
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable

In [8]:
simple_module = SimpleModule(name='simple')

In [9]:
simple_module(tf.constant(5.))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

### Example of a two-layer linear layer model made out of modules.

First a dense (linear) layer:

In [16]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y) 

In [17]:
class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)
        
    def __call__(self, x):
        return self.dense_2(self.dense_1(x))
        

In [18]:
my_model = SequentialModule(name='my_module')
print('Model results: ', my_model(tf.constant([ [2., 2., 2.] ])))
print('Submodules: ', my_model.submodules)
for var in my_model.variables:
    print(var)

Model results:  tf.Tensor([[0.       8.417499]], shape=(1, 2), dtype=float32)
Submodules:  (<__main__.Dense object at 0x000001DAC10AC908>, <__main__.Dense object at 0x000001DAC10AC8C8>)
<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>
<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.5442032 , -1.6187414 ,  1.4587874 ],
       [-0.20054685, -0.82662535, -1.0183262 ],
       [ 2.010697  ,  1.5654202 , -1.0852054 ]], dtype=float32)>
<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>
<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.04522573,  1.7876456 ],
       [ 0.6071877 , -1.6608255 ],
       [-0.50740117,  0.91678923]], dtype=float32)>


### Example of a two-layer linear layer model made out of modules with unknown input dimension.

In [19]:
class FlexibleDenseModule(tf.Module):
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features
        
    def __call__(self, x):
        if self.is_built != True:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True
        
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)
    

In [20]:
class MyFlexibleSequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [21]:
my_model = MyFlexibleSequentialModule(name="the_model")
print("Flexible Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Flexible Model results: tf.Tensor([[6.5807467 0.       ]], shape=(1, 2), dtype=float32)


### Saving weights
You can save a tf.Module as both a checkpoint and a SavedModel.

Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules):

In [ ]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

You can look inside a checkpoint to be sure the whole collection of variables is saved, sorted by the Python object that contains them.

In [ ]:
tf.train.list_variables(chkp_path)

When you load models back in, you overwrite the values in your Python object.

In [ ]:
new_model = SequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

### Saving functions

In [22]:
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [ ]:
tf.saved_model.save(my_model, "the_saved_model")
new_model = tf.saved_model.load("the_saved_model")

### Keras models and layers